This is the required imports. We use the requests to request data form the API and we import the pandas and json_normalize to get better vision of the data, which makes it more easy to analyze from

In [9]:
import requests
import pandas as pd
from pandas import json_normalize

In [10]:
# This is the API-token that we got from sportmonks on the "free" subscription.
# This should give us data from the danish superliga and the scottish first league
API_TOKEN = "KuQ4Rt1ypOCvXfcm3cZXdPhOUlbuOrpHBgJkFm1MWTvtRR8TLhgEI02hjDxz"

In [3]:
#Lets see if that works by loading the leagues from the free plan

leagues = requests.get("https://api.sportmonks.com/v3/football/odds?api_token="+API_TOKEN)

leagues = leagues.json()

In [4]:
#Lets see the results
leagues

{'message': 'The requested endpoint does not exist'}

In [6]:
base_url = "https://api.sportmonks.com/v3/football/odds"
endpoint = "/pre-match"
include = "market;bookmaker;fixture"
endpoint_url = f"{base_url}{endpoint}?include={include}&api_token={API_TOKEN}"

response = requests.get(
    endpoint_url,
    params={"api_token": API_TOKEN, "include": include, "per_page": 100, "page": 1}
)

data = response.json()

rows = []
for odd in data.get("data", []):
    market = odd.get("market") or {}
    bookmaker = odd.get("bookmaker") or {}
    fixture = odd.get("fixture") or {}
    rows.append([
        odd.get("id"),
        market.get("name"),
        bookmaker.get("name"),
        fixture.get("name") or fixture.get("id"),
    ])

df = pd.DataFrame(rows, columns=["OddID", "Market", "Bookmaker", "Fixture"])
df = df[df["Bookmaker"].str.contains("bet", case=False, na=False)]
df

,OddID,Market,Bookmaker,Fixture
0,181189,Fulltime Result,10Bet,Nordsjælland vs SønderjyskE
1,181190,Fulltime Result,10Bet,Nordsjælland vs SønderjyskE
2,181191,Fulltime Result,10Bet,Nordsjælland vs SønderjyskE
3,181192,Fulltime Result,BetFred,Nordsjælland vs SønderjyskE
4,181193,Fulltime Result,BetFred,Nordsjælland vs SønderjyskE
5,181194,Fulltime Result,BetFred,Nordsjælland vs SønderjyskE
6,181195,Fulltime Result,Bet-At-Home,Nordsjælland vs SønderjyskE
7,181196,Fulltime Result,Bet-At-Home,Nordsjælland vs SønderjyskE
8,181197,Fulltime Result,Bet-At-Home,Nordsjælland vs SønderjyskE
9,181198,Fulltime Result,Betfair,Nordsjælland vs SønderjyskE


In [11]:
import requests
import pandas as pd

API_TOKEN = "KuQ4Rt1ypOCvXfcm3cZXdPhOUlbuOrpHBgJkFm1MWTvtRR8TLhgEI02hjDxz"

ROUND_ID = 373227  # <-- put your round id here (example from your screenshot)

base_url = "https://api.sportmonks.com/v3/football"
endpoint = f"/rounds/{ROUND_ID}"

# Matches the screenshot
include = "fixtures.odds.market;fixtures.odds.bookmaker;fixtures.participants;league.country"
filters = "markets:1;bookmakers:2"

response = requests.get(
    f"{base_url}{endpoint}",
    params={
        "api_token": API_TOKEN,
        "include": include,
        "filters": filters,
    },
    timeout=30,
)

response.raise_for_status()
data = response.json()

# --- parse: round -> fixtures[] -> odds[] ---
round_data = data.get("data", {}) or {}
fixtures = round_data.get("fixtures", []) or []

rows = []
for fx in fixtures:
    fixture_name = fx.get("name") or str(fx.get("id"))
    starting_at = fx.get("starting_at")

    odds = fx.get("odds", []) or []
    for odd in odds:
        market = odd.get("market") or {}
        bookmaker = odd.get("bookmaker") or {}

        rows.append(
            {
                "Fixture": fixture_name,
                "StartingAt": starting_at,
                "OddID": odd.get("id"),
                "MarketID": odd.get("market_id"),
                "Market": market.get("name"),
                "BookmakerID": odd.get("bookmaker_id"),
                "Bookmaker": bookmaker.get("name"),
                "Label": odd.get("label"),   # e.g., Home/Draw/Away (depends on market)
                "Odds": odd.get("value"),    # decimal odds as string/number
            }
        )

df = pd.DataFrame(rows)

df


,Fixture,StartingAt,OddID,MarketID,Market,BookmakerID,Bookmaker,Label,Odds
0,Mallorca vs Girona,2026-01-04 17:30:00,209275551981,1,Fulltime Result,2,bet365,Away,3.60
1,Mallorca vs Girona,2026-01-04 17:30:00,209275551979,1,Fulltime Result,2,bet365,Draw,3.10
2,Mallorca vs Girona,2026-01-04 17:30:00,209275551977,1,Fulltime Result,2,bet365,Home,2.20
3,Rayo Vallecano vs Getafe,2026-01-02 20:00:00,209270514400,1,Fulltime Result,2,bet365,Away,4.33
4,Rayo Vallecano vs Getafe,2026-01-02 20:00:00,209270514399,1,Fulltime Result,2,bet365,Draw,2.87
5,Rayo Vallecano vs Getafe,2026-01-02 20:00:00,209270514398,1,Fulltime Result,2,bet365,Home,2.10
6,Elche vs Villarreal,2026-01-03 17:30:00,209270545293,1,Fulltime Result,2,bet365,Away,2.25
7,Elche vs Villarreal,2026-01-03 17:30:00,209270545290,1,Fulltime Result,2,bet365,Draw,3.40
8,Elche vs Villarreal,2026-01-03 17:30:00,209270545282,1,Fulltime Result,2,bet365,Home,3.10
9,Celta de Vigo vs Valencia,2026-01-03 13:00:00,209270545050,1,Fulltime Result,2,bet365,Away,3.90
